In [1]:
import numpy as np
import datetime as dt
import pandas as pd
import pyspedas
from pytplot import del_data, get_data, get_timespan, store_data, tplot_options, tplot_names, tplot, tplot_math

In [66]:
def get_omni(start, stop, fixed):
    if fixed:
        start = start+dt.timedelta(minutes=30)

    trange = [start.strftime("%Y-%m-%d/%H:%M:%S"), stop.strftime("%Y-%m-%d/%H:%M:%S")] #Timeseries should be in the format [pd.Timestamp, pd.Timestamp]
    print(trange)
    variables = ['BX_GSE', 'BY_GSE', 'BZ_GSE', 'Vx', 'Vy', 'Vz', 'proton_density', 'T']

    omni_import = pyspedas.omni.data(trange=trange, datatype='1min', level='hro2', time_clip=True)
    for n in variables:
        tplot_math.interp_nan(n)
    omni_data = pd.DataFrame([get_data('IMF')[0], get_data('BX_GSE')[1], get_data('BY_GSE')[1], get_data('BZ_GSE')[1], get_data('Vx')[1], get_data('Vy')[1], get_data('Vz')[1], get_data('proton_density')[1], get_data('T')[1]]).transpose()
    omni_data.columns = ['Time', 'BX_GSE', 'BY_GSE', 'BZ_GSE', 'Vx', 'Vy', 'Vz', 'proton_density', 'T']
    omni_data['Time'] = pd.to_datetime(omni_data['Time'], unit='s')

    return omni_data

In [67]:
def import_events(path, satellite1, satellite2): # Function to import the data from a satellite to compare with Artemis: either OMNI, THEMIS, or MMS
    events = pd.read_csv(path, delimiter=',', header=0)
    events['time_start'] = pd.to_datetime(events['time_start'])
    events['time_stop'] = pd.to_datetime(events['time_stop'])

    if satellite2 == 'omni':
        sat2_data = get_omni(events['time_start'][0], events['time_stop'][0], fixed=True)


    return sat2_data

In [68]:
data = import_events('../eventlist/eventlist.csv', 'themis', 'omni')

07-Mar-23 16:07:35: Downloading remote index: https://spdf.gsfc.nasa.gov/pub/data/omni/omni_cdaweb/hro2_1min/2011/


['2011-10-24/18:40:00', '2011-10-24/23:30:00']


07-Mar-23 16:07:35: File is current: omni_data/hro2_1min/2011/omni_hro2_1min_20111001_v01.cdf
07-Mar-23 16:07:36: Time clip was applied to: IMF
07-Mar-23 16:07:36: Time clip was applied to: PLS
07-Mar-23 16:07:36: Time clip was applied to: IMF_PTS
07-Mar-23 16:07:36: Time clip was applied to: PLS_PTS
07-Mar-23 16:07:36: Time clip was applied to: percent_interp
07-Mar-23 16:07:36: Time clip was applied to: Timeshift
07-Mar-23 16:07:36: Time clip was applied to: RMS_Timeshift
07-Mar-23 16:07:36: Time clip was applied to: RMS_phase
07-Mar-23 16:07:36: Time clip was applied to: Time_btwn_obs
07-Mar-23 16:07:36: Time clip was applied to: F
07-Mar-23 16:07:36: Time clip was applied to: BX_GSE
07-Mar-23 16:07:36: Time clip was applied to: BY_GSE
07-Mar-23 16:07:36: Time clip was applied to: BZ_GSE
07-Mar-23 16:07:36: Time clip was applied to: BY_GSM
07-Mar-23 16:07:36: Time clip was applied to: BZ_GSM
07-Mar-23 16:07:36: Time clip was applied to: RMS_SD_B
07-Mar-23 16:07:36: Time clip was app

In [69]:
data

,Time,BX_GSE,BY_GSE,BZ_GSE,Vx,Vy,Vz,proton_density,T
0,2011-10-24 18:40:00,3.37,-3.490000,-6.83,-386.399994,-1.600000,-15.000000,17.299999,138819.0
1,2011-10-24 18:41:00,2.40,-0.950000,-8.42,-350.799988,-4.500000,-0.600000,9.840000,103815.0
2,2011-10-24 18:42:00,2.30,-0.960000,-6.63,-374.500000,-4.300000,-7.900000,12.050000,138498.0
3,2011-10-24 18:43:00,3.76,-7.450000,-7.60,-487.299988,-5.400000,-34.700001,28.520000,341575.0
4,2011-10-24 18:44:00,2.65,-8.260000,0.04,-449.100006,-10.700000,-9.100000,22.209999,298829.0
...,...,...,...,...,...,...,...,...,...
286,2011-10-24 23:26:00,8.33,14.400000,-5.78,-491.600006,58.099998,-30.400000,25.840000,212031.0
287,2011-10-24 23:27:00,7.30,15.120000,-6.75,-500.700012,57.599998,-32.400002,24.250000,200028.0
288,2011-10-24 23:28:00,4.79,16.280001,-4.67,-502.299988,60.200001,-25.400000,24.379999,188880.0
289,2011-10-24 23:29:00,6.82,15.300000,-5.39,-503.600006,62.299999,-19.700001,24.490000,179700.0


In [ ]:















    probe = [] # Initialize probe ID array
    bowshock_times = [] # Initialize start and stop time array for 'satellite'
    lunar_times = [] # Initialize start and stop time array for ARTEMIS

    with open(path, newline='') as events:
        rows = csv.reader(events)
        next(rows)
        for r in rows:
            bowshock_times.append([(dt.datetime.strptime(r[0], '%Y-%m-%dT%H:%M:%S.%fZ')+dt.timedelta(minutes=30)).strftime('%Y-%m-%d/%H:%M:%S'), (dt.datetime.strptime(r[1], '%Y-%m-%dT%H:%M:%S.%fZ')).strftime('%Y-%m-%d/%H:%M:%S')])
            lunar_times.append([(dt.datetime.strptime(r[0], '%Y-%m-%dT%H:%M:%S.%fZ')).strftime('%Y-%m-%d/%H:%M:%S'), (dt.datetime.strptime(r[1], '%Y-%m-%dT%H:%M:%S.%fZ')).strftime('%Y-%m-%d/%H:%M:%S')])
            if satellite == 'mms':
                probe.append(r[2])
            elif satellite == 'themis':
                probe.append(str(r[3]))
            elif satellite == 'omni':
                probe.append('o')

        if satellite == 'omni':
            keys = np.arange(len(probe)) # Create dictionary keys 0...n equal to the number of input events
            types = ('time', 'bx', 'by', 'bz', 'vx', 'vy', 'vz', 'n', 'T') # Keys for the sub-dictionary in each event
            data = dict((i, dict.fromkeys(types)) for i in keys) # Create the 'data' dictionary
            for i in keys: # Iterate through every event
                import_data = pyspedas.omni.data(trange=bowshock_times[i], datatype='1min', level='hro2', time_clip=True) # Import the tplot variable for the ith event
                varnames = ['BX_GSE', 'BY_GSE', 'BZ_GSE', 'Vx', 'Vy', 'Vz', 'proton_density', 'T']
                for p in varnames:
                    tplot_math.interp_nan(p)
                products = [get_data('IMF')[0], get_data('BX_GSE')[1], get_data('BY_GSE')[1], get_data('BZ_GSE')[1], get_data('Vx')[1], get_data('Vy')[1], get_data('Vz')[1], get_data('proton_density')[1], get_data('T')[1]] # Each event in the 'data' dictionary will consist of these products
                for j, k in enumerate(types): # Now go through the keys of the sub-dictionary
                    data[i][k] = products[j] # For the first sub-dict key, set that equal to the first product (and so on)...

                data[i]['time'] = data[i]['time'].astype('object')
                for n in range(len(data[i]['time'])):
                    data[i]['time'][n] = dt.datetime.utcfromtimestamp(data[i]['time'][n])


        if satellite == 'themis':
            keys = np.arange(len(probe)) # Create dictionary keys 0...n equal to the number of input events
            fgm_types = ('time', 'bx', 'by', 'bz') # Keys for the sub-dictionary in each event
            esa_types = ('time', 'vx', 'vy', 'vz', 'n', 'T')
            themis_fgm_data = dict((i, dict.fromkeys(fgm_types)) for i in keys) # Create the 'fgm_data' dictionary
            themis_esa_data = dict((i, dict.fromkeys(esa_types)) for i in keys) # Create the 'esa_data' dictionary
            for i in keys:
                themis_fgm_import = pyspedas.themis.fgm(trange=bowshock_times[i], probe=probe[i], time_clip=True, varnames='th'+probe[i]+'_fgs_gse')
                themis_esa_import = pyspedas.themis.esa(trange=bowshock_times[i], probe=probe[i], time_clip=True, varnames=['th'+probe[i]+'_peif_density', 'th'+probe[i]+'_peif_avgtemp', 'th'+probe[i]+'_peif_velocity_gse'])
                fgm_products = [get_data('th'+probe[i]+'_fgs_gse')[0], get_data('th'+probe[i]+'_fgs_gse')[1][:,0], get_data('th'+probe[i]+'_fgs_gse')[1][:,1], get_data('th'+probe[i]+'_fgs_gse')[1][:,2]]
                esa_products = [get_data('th'+probe[i]+'_peif_velocity_gse')[0], get_data('th'+probe[i]+'_peif_velocity_gse')[1][:,0], get_data('th'+probe[i]+'_peif_velocity_gse')[1][:,1], get_data('th'+probe[i]+'_peif_velocity_gse')[1][:,2], get_data('th'+probe[i]+'_peif_density')[1], get_data('th'+probe[i]+'_peif_avgtemp')[1]]
                for j, k in enumerate(fgm_types):
                    themis_fgm_data[i][k] = fgm_products[j]
                for m, n in enumerate(esa_types):
                    themis_esa_data[i][n] = esa_products[m]

        keys = np.arange(len(probe)) # Create dictionary keys 0...n equal to the number of input events
        fgm_types = ('time', 'bx', 'by', 'bz') # Keys for the sub-dictionary in each event
        esa_types = ('time', 'vx', 'vy', 'vz', 'n', 'T')
        artemis_fgm_data = dict((i, dict.fromkeys(fgm_types)) for i in keys) # Create the 'fgm_data' dictionary
        artemis_esa_data = dict((i, dict.fromkeys(esa_types)) for i in keys) # Create the 'esa_data' dictionary
        for i in keys:
            artemis_fgm_import = pyspedas.themis.fgm(trange=lunar_times[i], probe='b', time_clip=True, varnames='thb_fgs_gse')
            artemis_esa_import = pyspedas.themis.esa(trange=lunar_times[i], probe='b', time_clip=True, varnames=['thb_peif_density', 'thb_peif_avgtemp', 'thb_peif_velocity_gse'])
            fgm_products = [get_data('thb_fgs_gse')[0], get_data('thb_fgs_gse')[1][:,0], get_data('thb_fgs_gse')[1][:,1], get_data('thb_fgs_gse')[1][:,2]]
            esa_products = [get_data('thb_peif_velocity_gse')[0], get_data('thb_peif_velocity_gse')[1][:,0], get_data('thb_peif_velocity_gse')[1][:,1], get_data('thb_peif_velocity_gse')[1][:,2], get_data('thb_peif_density')[1], get_data('thb_peif_avgtemp')[1]]
            for j, k in enumerate(fgm_types):
                artemis_fgm_data[i][k] = fgm_products[j]
            for j, k in enumerate(esa_types):
                artemis_esa_data[i][k] = esa_products[j]

            artemis_fgm_data[i]['time'] = artemis_fgm_data[i]['time'].astype('object')
            artemis_esa_data[i]['time'] = artemis_esa_data[i]['time'].astype('object')
            for n in range(len(artemis_fgm_data[i]['time'])):
                artemis_fgm_data[i]['time'][n] = dt.datetime.utcfromtimestamp(artemis_fgm_data[i]['time'][n])
            for n in range(len(artemis_esa_data[i]['time'])):
                artemis_esa_data[i]['time'][n] = dt.datetime.utcfromtimestamp(artemis_esa_data[i]['time'][n])



    if satellite == 'omni':
        return data, artemis_fgm_data, artemis_esa_data
    if satellite == 'themis':
        return themis_fgm_data, themis_esa_data, artemis_fgm_data, artemis_esa_data